<a href="https://colab.research.google.com/github/bengisudurugoksu/AnomalyDetectioninTimeSeries/blob/main/finance_preprocessing_bengi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Eğer 'Datetime' görünmüyorsa index'ten çıkaralım
if not 'Datetime' in data.columns:
    data = data.reset_index()  # index'i sütuna çevirir

# Multi-index kolon adlarını düzleştir
data.columns = ['_'.join([str(c) for c in col]).strip() if isinstance(col, tuple) else str(col) for col in data.columns]

# Şu anda hangi sütunlar var bir kontrol edelim
print("📊 Sütun isimleri:", list(data.columns)[:15])

# Şimdi tabloyu yeniden biçimlendirelim
symbols = ["AAPL", "TSLA", "MSFT"]
dfs = []
for s in symbols:
    # Her sembol için kolon adlarını kontrol et (bazı versiyonlarda büyük harf yerine küçük olabiliyor)
    possible_cols = [col for col in data.columns if s in col]
    print(f"\n✅ {s} ile ilgili kolonlar:", possible_cols)

    # Eğer Datetime kolonumuz hâlâ varsa kullan
    time_col = [c for c in data.columns if "Date" in c or "Datetime" in c][0]

    subset = data[[time_col, f'Close_{s}', f'High_{s}', f'Low_{s}', f'Open_{s}', f'Volume_{s}']].copy()
    subset.columns = ['Datetime', 'Close', 'High', 'Low', 'Open', 'Volume']
    subset['Symbol'] = s
    dfs.append(subset)

# Hepsini birleştir
df = pd.concat(dfs, ignore_index=True)
print("\nTemiz tablo boyutu:", df.shape)
df.head()


📊 Sütun isimleri: ['level_0', 'index', 'index_', 'Datetime_', 'Close_AAPL', 'High_AAPL', 'Low_AAPL', 'Open_AAPL', 'Volume_AAPL', 'Symbol_', 'Close_TSLA', 'High_TSLA', 'Low_TSLA', 'Open_TSLA', 'Volume_TSLA']

✅ AAPL ile ilgili kolonlar: ['Close_AAPL', 'High_AAPL', 'Low_AAPL', 'Open_AAPL', 'Volume_AAPL']

✅ TSLA ile ilgili kolonlar: ['Close_TSLA', 'High_TSLA', 'Low_TSLA', 'Open_TSLA', 'Volume_TSLA']

✅ MSFT ile ilgili kolonlar: ['Close_MSFT', 'High_MSFT', 'Low_MSFT', 'Open_MSFT', 'Volume_MSFT']

Temiz tablo boyutu: (15768, 7)


,Datetime,Close,High,Low,Open,Volume,Symbol
0,2024-01-02 14:30:00+00:00,186.189896,188.440002,186.009995,187.800995,21173291.0,AAPL
1,2024-01-02 15:30:00+00:00,186.804993,186.904999,185.184296,186.179993,9010595.0,AAPL
2,2024-01-02 16:30:00+00:00,186.615005,187.350006,186.570007,186.800003,6143754.0,AAPL
3,2024-01-02 17:30:00+00:00,185.294998,187.039993,185.259995,186.615005,5381701.0,AAPL
4,2024-01-02 18:30:00+00:00,184.494995,185.539993,184.229996,185.289993,7228643.0,AAPL


In [ ]:
import numpy as np

# Her hisse (Symbol) için log-return ve volatility hesapla
df['log_return'] = df.groupby('Symbol')['Close'].transform(lambda x: np.log(x / x.shift(1)))
df['volatility'] = df.groupby('Symbol')['log_return'].transform(lambda x: x.rolling(30).std())

# NaN değerleri temizle
df.dropna(inplace=True)

print("Yeni tablo boyutu:", df.shape)
df.head()


Yeni tablo boyutu: (5166, 9)


,Datetime,Close,High,Low,Open,Volume,Symbol,log_return,volatility
30,2024-01-08 16:30:00+00:00,183.990097,184.570007,183.235001,183.244995,5720535.0,AAPL,0.004058,0.005093
31,2024-01-08 17:30:00+00:00,184.115005,184.470001,183.960007,184.000000,4151910.0,AAPL,0.000679,0.005050
32,2024-01-08 18:30:00+00:00,184.832504,184.875000,184.050003,184.119995,3724042.0,AAPL,0.003889,0.005111
33,2024-01-08 19:30:00+00:00,185.220001,185.419998,184.699997,184.839996,5370765.0,AAPL,0.002094,0.004964
34,2024-01-08 20:30:00+00:00,185.550003,185.600006,184.910004,185.214996,6759299.0,AAPL,0.001780,0.004906


In [ ]:
import os
data_path = "/content/drive/MyDrive/anomaly_project/data"
os.makedirs(data_path, exist_ok=True)
df.to_csv(os.path.join(data_path, "yfinance_clean.csv"), index=False)
print("✅ Kaydedildi:", os.path.join(data_path, "yfinance_clean.csv"))


✅ Kaydedildi: /content/drive/MyDrive/anomaly_project/data/yfinance_clean.csv
